# Environment Set Up

In [ ]:
# G drive connection
# learn: to access a shared file just put a shortcut in mi drive

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Libraries import

import pandas as pd

# Connection test

In [ ]:
# Verify reading folder

!ls '/content/drive/MyDrive/andresrokp/Thesis Juandavid/'

In [ ]:
# Hola mundo

file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Companies.dta'
df = pd.read_stata(file_path)
print('\n---\ninfo\n---')
df.info()  # Show column
print('\n---\ndescribe\n---')
print(df.describe())  # Show very basic stadistics
print('\n---\nhead\n---')
df.head()  # Show first rows


# Dataset loading

In [ ]:
# File paths
path_companies_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Companies.dta'
path_ipo_m_a_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/IPO_M&A.dta'
path_investments_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investments.dta'
path_investors_file = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investors.dta'
path_vico = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/VICO5.0_Investors_Reputation_withOldIDs.dta'

# Read Companies data
print("Reading Companies data...")
df_companies = pd.read_stata(path_companies_file)
print("\nCompanies Data Info:")
print(df_companies.info())

# Read IPO M&A data
print("\nReading IPO M&A data...")
df_ipo_ma = pd.read_stata(path_ipo_m_a_file)
print("\nIPO M&A Data Info:")
print(df_ipo_ma.info())

# Read Investments data
print("\nReading Investments data...")
df_investments = pd.read_stata(path_investments_file)
print("\nInvestments Data Info:")
print(df_investments.info())

# Read Investors data
print("\nReading Investors data...")
df_investors = pd.read_stata(path_investors_file)
print("\nInvestors Data Info:")
print(df_investors.info())

# Read VICO5 data
print("\nReading vico5_BNT data...")
df_vico5_BNT = pd.read_stata(path_vico)
print("\nVICO5 Data Info:")
print(df_vico5_BNT.info())
print("Number of Rows in vico5_BNT Data:", len(df_vico5_BNT))

In [ ]:
'''
1st model: Wahal, (2004). Sorensen, (2007)

Used tables:
'df_investors',
'df_vico5_BNT'
'''

# Left JOIN de las tablas
# df_vico5_BNT >>InvestorID>> df_investors
merged_investments_investors = pd.merge(df_vico5_BNT, df_investors, on='InvestorID', how='left')

print('-\n--\n-')

# cleaning dejar solo VCs
df_Investments_merge_clean = merged_investments_investors[merged_investments_investors['InvestorType'] != 'BA']
df_Investments_merge_clean = df_Investments_merge_clean[df_Investments_merge_clean['InvestorType'] != 'Other']
df_Investments_merge_clean = df_Investments_merge_clean[~df_Investments_merge_clean['InvestorID'].str.contains('missing')]
print( 'df_Investments_merge_clean.info()', df_Investments_merge_clean.info() )
print( 'len (df_Investments_merge_clean)', len(df_Investments_merge_clean) )
# print(df_Investments_no_BAs.head(5))


col_InvestorID_clean = df_Investments_merge_clean['InvestorID']
# col_InvestorID_clean = pd.merge(df_Investments_bnt, df_Investors, on='InvestorID', how='left')['InvestorName']
# print(df_Investments_clean.head(5))

numberOf_Unique_InvestorsClean = len(col_InvestorID_clean.unique())
print( 'numberOf_Unique_InvestorsClean', numberOf_Unique_InvestorsClean )

uniqueInvestorsAppereanceCount = col_InvestorID_clean.value_counts()
print('uniqueInvestorsAppereanceCount\n', uniqueInvestorsAppereanceCount)
print( 'len( uniqueInvestorsAppereanceCount )', len( uniqueInvestorsAppereanceCount ) )
print( 'uniqueInvestorsAppereanceCount', uniqueInvestorsAppereanceCount )


print("\n\nInvestorName_Final conteo de apariciones TOP 25%")
col_InvestorID_clean.value_counts().head(round(numberOf_Unique_InvestorsClean*0.25))

In [ ]:
# Exploración de tipo VC

file_path = '/content/drive/MyDrive/andresrokp/Thesis Juandavid/Investors.dta'
df_Investors = pd.read_stata(file_path)

print('\n---\ninfo\n---')
df_Investors.info()  # Show column
print('\n---\ndescribe\n---')
print(df_Investors.describe())  # Show very basic stadistics
print('\n---\nhead\n---')
df_Investors.head()  # Show first rows

col_investor_name = df_Investors['InvestorType']


# print("\n\nInvestorName_Final missing_values")
# print( col_investor_name.isnull().sum() )

# print("\n\nInvestorName_Final unique inversores")
# print( col_investor_name.unique() )
# numberOfUniqueInvestors = len(col_investor_name.unique())
# print( numberOfUniqueInvestors )

# esto es para comparar loque dice el pdf en cantidad de BA's y VC'c
print("\n\nInvestorName_Final conteo de apariciones tipo de inversor")
print( len(col_investor_name.value_counts() ) )
print( col_investor_name.value_counts() )

In [ ]:
# Check runtime vars
# dir() keeps trak of the declared variables in the runtime

[var for var in dir() if '_' != var[0]]